The purpose of autonomously adjusting reference voltage (AARV) is to mitigate rapid voltage change (RVC), without affecting the utility’s control of steady-state voltage or the DER’s steady-state reactive power.  AARV may be used with a volt-watt function that mitigates steady-state overvoltage, but otherwise AARV does not regulate steady-state voltage.

Page 39 of IEEE 1547-2018 says: _The DER shall be capable of autonomously adjusting reference voltage (VRef) with VRef being equal to the low pass filtered measured voltage. The time constant shall be adjustable at least over the range of 300 s to 5000 s. The voltage-reactive power Volt-Var curve characteristic shall be adjusted autonomously as VRef changes. The approval of the Area EPS operator shall be required for the DER to autonomously adjust the reference voltage. Implementation of the autonomous VRef adjustability and the associated time constant shall be specified by the Area EPS operator._

Clause 5.14.5 of IEEE 1547.1-2020 describes a test for AARV. In all other tests, AARV is to be disabled. Four AARV tests are described, at the time constant Tref = 300s and 5000s, each of those with source voltage steps to (V1+V2)/2 and (V3+V4)/2. Before each voltage step, the steady-state reactive power, Q, should be zero at V=Vref. The voltage steps result in step changes of Q to 0.5*Q1 and 0.5*Q4, respectively, with some delay from the open-loop response. These new Q values should follow an exponential decay back to zero, governed by Tref = 300s or 5000s. The test criteria specifies that after one time constant, Tref, the value of Q should be less than 10% of Q1 or Q4, respectively. However, the expected value of Q, following an exponential decay from the stepped value, is Q = 0.5*Q1*exp(-Tref/Tref) = 0.1839*Q1 or 0.1839*Q4. The evaluation criteria at 1*Tref should be 20% instead of 10%.  Note: at 2*Tref, the expected value of Q is 0.0677*Q1 or 0.0677*Q4. This test verifies implementation of Tref, but not the impact of AARV on a system with grid impedance.

2019 Conference Paper: https://doi.org/10.1109/PVSC40753.2019.8981277 

2023 Conference Paper: https://doi.org/10.1109/PESGM52003.2023.10252317

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math
plt.rcParams['savefig.directory'] = os.getcwd()

Run the following cells to generate the plots using [Matplotlib](https://matplotlib.org/)

In [ ]:
dt = 0.1
Tref = 300.0
Tresponse = 5.0
TauOL = Tresponse / 2.3026
IncRef = 1.0 - math.exp(dt/Tref)
IncOL = 1.0 - math.exp(dt/TauOL)

#Verr = Vpu - Vreg
#Vreg = Vreg + Verr * IncRef
#Qtarg = -K * (V - Vreg) + Qbias # differs from OpenDSS in that it does not iterate
#Q = Qlast + (Q - Qlast) * IncOL

# plot P, Q, V, Vreg, Vsrc
# input Vreg, K, dB, Q1, Q4
# calculate Q2 = Q3 = 0
Vreg = 1.0
dB = 0.02
K = 22.0
Q1 = 0.44
Q2 = 0.0
Q3 = 0.0
Q4 = -0.44
V2 = Vreg - 0.5 * dB
V3 = Vreg + 0.5 * dB
V1 = V2 - Q1 / K
V4 = V3 - Q4 / K
VL = min(V1, 0.95)
VH = max(V4, 1.05)
VTABLE = np.array ([VL, V1, V2, V3, V4, VH])
QTABLE = np.array ([Q1, Q1, Q2, Q3, Q4, Q4])
VMIN = VL - 0.01
VMAX = VH + 0.01
print ('Q = [{:7.4f},{:7.4f},{:7.4f},{:7.4f}]'.format (Q1, Q2, Q3, Q4)) 
print ('V = [{:7.4f},{:7.4f},{:7.4f},{:7.4f}]'.format (V1, V2, V3, V4))

v = np.linspace (VMIN, VMAX, 501)
q = np.interp (v, VTABLE, QTABLE)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 1, sharex = 'col', figsize=(6,4), constrained_layout=True)
fig.suptitle ('Volt-var characteristic for Vreg={:.3f}, dB={:.3f}, K={:.2f}'.format (Vreg, dB, K))
ax.plot(VTABLE, QTABLE, marker='o', color='blue', label='Points')
ax.plot (v, q, color='red', label='Interpolated')
ax.grid()
ax.set_xlabel ('V [pu]')
ax.set_ylabel ('Q [pu]')
ax.set_xlim (VMIN, VMAX)
ax.legend()

plt.show()

In [ ]:
# approximate voltage change for the Nantucket BESS
Vt = 13200.0
R = 1.210 # 1.744
X = 2.8339 # 2.6797
dP = 6.0e6
dQ = 0.0
a1 = Vt*Vt + R*dP + X*dQ
a2 = X*dP - R*dQ
d = math.sqrt (a1*a1 + a2*a2) / Vt / Vt - 1.0
print ('d = {:.4f} pu'.format (d))